## Importing libraries

In [3]:
import os
import random
import itertools
import copy
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from tqdm import trange
from PIL import Image
from collections import Counter

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms



def contrastive_loss_function(x0, x1, y, margin=1.0):
    # euclidean distance
    diff = x0 - x1
    dist_sq = torch.sum(torch.pow(diff, 2), 1)
    dist = torch.sqrt(dist_sq)
    mdist = margin - dist
    dist = torch.clamp(mdist, min=0.0)
    loss = y * dist_sq + (1 - y) * torch.pow(dist, 2)
    loss = torch.sum(loss) / 2.0 / x0.size()[0]
    return loss

# Loading and preparing data

In [6]:
#data_dir = './data/processed/'
back_dir = './data/background/'
eval_dir = './data/evaluation/'
write_dir = './data/changed/'

if not os.path.exists(write_dir):
    os.makedirs(write_dir)

In [5]:
# ensuring reproducibility
np.random.seed(0)
random.seed(0)

In [7]:
# params
percentage_split = False
augment = False

In [12]:
background_alphabets = [os.path.join(back_dir, x) for x in next(os.walk(back_dir))[1]]
background_alphabets.sort()

# list of all drawers (1 to 20)
background_drawers = list(np.arange(1, 21))

print("There are {} alphabets.".format(len(background_alphabets)))

There are 40 alphabets.


In [13]:
# from 40 alphabets, randomly select 30
train_alphabets = list(np.random.choice(background_alphabets, size=30, replace=False))
valid_alphabets = [x for x in background_alphabets if x not in train_alphabets]

train_alphabets.sort()
valid_alphabets.sort()

In [15]:
train_write = os.path.join(write_dir, 'train')

for alphabet in train_alphabets:
    train_write_1 = train_write + '/' + alphabet.split('/')[-1] + '_' 
    for char in os.listdir(alphabet):
        train_write_2 = train_write_1 + char
        char_path = os.path.join(alphabet, char)
        os.makedirs(train_write_2)
        for drawer in os.listdir(char_path):
            drawer_path = os.path.join(char_path, drawer)
            shutil.copyfile(
                drawer_path, os.path.join(
                    train_write_2, drawer
                )
            )

In [16]:
valid_write = os.path.join(write_dir, 'valid')

for alphabet in valid_alphabets:
    valid_write_1 = valid_write + '/' + alphabet.split('/')[-1] + '_' 
    for char in os.listdir(alphabet):
        valid_write_2 = valid_write_1 + char
        char_path = os.path.join(alphabet, char)
        os.makedirs(valid_write_2)
        for drawer in os.listdir(char_path):
            drawer_path = os.path.join(char_path, drawer)
            shutil.copyfile(
                drawer_path, os.path.join(
                    valid_write_2, drawer
                )
            )

In [17]:
# get list of alphabets
test_alphabets = [os.path.join(eval_dir, x) for x in next(os.walk(eval_dir))[1]]
test_alphabets.sort()

In [19]:
test_write = os.path.join(write_dir, 'test')

for alphabet in test_alphabets:
    test_write_1 = test_write + '/' + alphabet.split('/')[-1] + '_' 
    for char in os.listdir(alphabet):
        test_write_2 = test_write_1 + char
        char_path = os.path.join(alphabet, char)
        os.makedirs(test_write_2)
        for drawer in os.listdir(char_path):
            drawer_path = os.path.join(char_path, drawer)
            shutil.copyfile(
                drawer_path, os.path.join(
                    test_write_2, drawer
                )
            )